In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ تحميل البيانات
df_master = pd.read_excel("Product Matching Dataset.xlsx")
df_seller = pd.read_excel("Product Matching Dataset2.xlsx")

# 2️⃣ تنظيف البيانات (إزالة المسافات الزائدة وتحويل النصوص لأحرف صغيرة)
df_seller["seller_item_name_clean"] = df_seller["seller_item_name"].str.strip().str.lower()
df_master["product_name_ar_clean"] = df_master["product_name_ar"].str.strip().str.lower()

# 3️⃣ تحويل الأسماء إلى قوائم نصوص
seller_names = df_seller["seller_item_name_clean"].fillna("").astype(str).tolist()
master_names = df_master["product_name_ar_clean"].fillna("").astype(str).tolist()

# 4️⃣ إنشاء TF-IDF Vectorizer (تحليل النصوص باستخدام n-grams)
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), max_features=5000)

# 5️⃣ تحويل الأسماء إلى مصفوفة رقمية
tfidf_matrix_seller = vectorizer.fit_transform(seller_names)
tfidf_matrix_master = vectorizer.transform(master_names)

# 6️⃣ تقسيم البيانات على دفعات صغيرة لتوفير الذاكرة
batch_size = 1000
num_batches = int(np.ceil(len(df_seller) / batch_size))
batch_results = []

for i in range(num_batches):
    start = i * batch_size
    end = (i + 1) * batch_size
    batch_seller_names = seller_names[start:end]

    # تحويل كل دفعة إلى مصفوفة TF-IDF
    tfidf_matrix_seller_batch = vectorizer.transform(batch_seller_names)

    # حساب تشابه كوساين فقط لهذه الدفعة
    cosine_similarities_batch = cosine_similarity(tfidf_matrix_seller_batch, tfidf_matrix_master)

    # استخراج أفضل تطابق
    best_matches_batch = cosine_similarities_batch.argmax(axis=1)
    best_scores_batch = cosine_similarities_batch.max(axis=1)

    # تخزين النتائج
    for j, seller_index in enumerate(range(start, end)):
        if seller_index >= len(df_seller):
            break  # نتأكد من عدم تخطي عدد المنتجات
        
        best_match_index = best_matches_batch[j]
        batch_results.append((
            df_seller.iloc[seller_index]["sku"], 
            df_seller.iloc[seller_index]["seller_item_name"], 
            df_master.iloc[best_match_index]["sku"], 
            df_master.iloc[best_match_index]["product_name_ar_clean"], 
            best_scores_batch[j]
        ))

# 7️⃣ تحويل النتائج إلى DataFrame
optimized_matches_df = pd.DataFrame(batch_results, columns=["seller_sku", "seller_item_name", "matched_sku", "matched_name", "similarity_score"])

# 8️⃣ دالة تصنيف مستوى الثقة
def get_confidence_level(score):
    if score >= 0.90:
        return "High"
    elif score >= 0.70:
        return "Medium"
    else:
        return "Low"

# 9️⃣ تطبيق التصنيف على النتائج
optimized_matches_df["confidence_level"] = optimized_matches_df["similarity_score"].apply(get_confidence_level)

# 🔟 حفظ النتيجة في ملف إكسل
output_file = "Product_Matching_Results.xlsx"
optimized_matches_df.to_excel(output_file, index=False)
print("\n✅ تم حفظ النتائج النهائية مع مستوى الثقة في 'Product_Matching_Results.xlsx'")



✅ تم حفظ النتائج النهائية مع مستوى الثقة في 'Product_Matching_Results.xlsx'


In [ ]:
""""
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time  # مكتبة الوقت لحساب زمن التنفيذ

# بدء تسجيل الوقت لحساب زمن التنفيذ
# start_time = time.time()

# تحميل ملفات البيانات
df_master = pd.read_excel("Product Matching Dataset.xlsx")
df_seller = pd.read_excel("Product Matching Dataset2.xlsx")

# التحقق من نجاح التحميل
# data_load_time = time.time()
# print(f"📥 تم تحميل البيانات في {data_load_time - start_time:.2f} ثوانٍ")

# تنظيف الأسماء بإزالة المسافات الزائدة وتحويل النصوص إلى حروف صغيرة
df_seller["seller_item_name_clean"] = df_seller["seller_item_name"].str.strip().str.lower()
df_master["product_name_ar_clean"] = df_master["product_name_ar"].str.strip().str.lower()

# انتهاء عملية تنظيف البيانات
# clean_data_time = time.time()
# print(f"🧹 تم تنظيف البيانات في {clean_data_time - data_load_time:.2f} ثوانٍ")

# تحويل أسماء المنتجات إلى قوائم نصوص لاستخدامها في التحليل
seller_names = df_seller["seller_item_name_clean"].fillna("").astype(str).tolist()
master_names = df_master["product_name_ar_clean"].fillna("").astype(str).tolist()

# إنشاء نموذج TF-IDF لمعالجة النصوص باستخدام n-grams
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), max_features=5000)

# تحويل الأسماء إلى مصفوفة TF-IDF
# tfidf_start_time = time.time()
tfidf_matrix_seller = vectorizer.fit_transform(seller_names)
tfidf_matrix_master = vectorizer.transform(master_names)

# انتهاء عملية التحويل
# tfidf_end_time = time.time()
# print(f"🔢 تحويل النصوص إلى TF-IDF تم في {tfidf_end_time - clean_data_time:.2f} ثوانٍ")

# تقسيم البيانات إلى دفعات لتوفير الذاكرة وتحسين الأداء
batch_size = 1000
num_batches = int(np.ceil(len(df_seller) / batch_size))
batch_results = []

# تسجيل وقت بدء عملية المطابقة
# match_start_time = time.time()

for i in range(num_batches):
    start = i * batch_size
    end = (i + 1) * batch_size
    batch_seller_names = seller_names[start:end]

    # تحويل الدفعة الحالية إلى TF-IDF
    tfidf_matrix_seller_batch = vectorizer.transform(batch_seller_names)

    # حساب تشابه كوساين لكل دفعة فقط
    cosine_similarities_batch = cosine_similarity(tfidf_matrix_seller_batch, tfidf_matrix_master)

    # استخراج أفضل تطابق من قاعدة البيانات
    best_matches_batch = cosine_similarities_batch.argmax(axis=1)
    best_scores_batch = cosine_similarities_batch.max(axis=1)

    # تخزين النتائج
    for j, seller_index in enumerate(range(start, end)):
        if seller_index >= len(df_seller):
            break  # التأكد من عدم تخطي عدد المنتجات
        
        best_match_index = best_matches_batch[j]
        batch_results.append((
            df_seller.iloc[seller_index]["sku"], 
            df_seller.iloc[seller_index]["seller_item_name"], 
            df_master.iloc[best_match_index]["sku"], 
            df_master.iloc[best_match_index]["product_name_ar_clean"], 
            best_scores_batch[j]
        ))

    # طباعة تقدم التنفيذ لكل دفعة
    # print(f"✅ تم تنفيذ الدفعة {i+1}/{num_batches} في {time.time() - match_start_time:.2f} ثوانٍ")

# تحويل النتائج إلى DataFrame لتحليلها لاحقًا
optimized_matches_df = pd.DataFrame(batch_results, columns=["seller_sku", "seller_item_name", "matched_sku", "matched_name", "similarity_score"])

# دالة لتحديد مستوى الثقة بناءً على نسبة التشابه
def get_confidence_level(score):
    if score >= 0.90:
        return "High"
    elif score >= 0.70:
        return "Medium"
    else:
        return "Low"

# تصنيف مستوى الثقة لكل منتج تم مطابقته
optimized_matches_df["confidence_level"] = optimized_matches_df["similarity_score"].apply(get_confidence_level)

# حفظ النتائج النهائية في ملف إكسل
output_file = "Product_Matching_Results.xlsx"
optimized_matches_df.to_excel(output_file, index=False)

# إنهاء عملية حفظ الملف
# save_time = time.time()
# print(f"📊 تم حفظ النتائج في {save_time - match_start_time:.2f} ثوانٍ")

# طباعة الزمن الكلي للتنفيذ
# end_time = time.time()
# print(f"\n🚀 الزمن الكلي لتنفيذ البرنامج: {end_time - start_time:.2f} ثانية")
